In [1]:
import os
import sys
import warnings

import pandas as pd
import numpy as np
import requests
from tqdm.auto import tqdm
from scipy.stats import mode
from pprint import pprint
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset.interactions import Interactions
from rectools.dataset import Dataset
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, MAP, calc_metrics

In [2]:
sys.path.append(os.path.abspath(os.path.join('../service/modelss/')))
from userknn import UserKnn

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
warnings.filterwarnings("ignore")

# Get KION dataset

<a href="https://ods.ai/competitions/competition-recsys-21/data"> Dataset description [ru] </a>


# EDA

In [4]:
interactions = pd.read_csv('../data/raw/kion_train/interactions.csv')
users = pd.read_csv('../data/raw/kion_train/users.csv')
items = pd.read_csv('../data/raw/kion_train/items.csv')

In [5]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

## interactions

In [5]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [25]:
print(f"Interactions dataframe shape: {interactions.shape}")
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}")

Interactions dataframe shape: (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [26]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


In [27]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       int64         
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


## users

In [28]:
pd.concat([users.head(), users.tail()])

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [29]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users dataframe shape (840197, 5)
Unique users: 840_197


## items

In [30]:
pd.concat([items.head(2), items.tail(2)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родди, Марк О’Халлоран, Джимми Смоллхорн","Семнадцатилетний Дэмиен мечтает вырваться за пределы своего района и стать профессиональным бойцом. Когда его кумир и старший брат исчезает, парень попадает в чуждый ему мир насилия, наркотиков и ...","Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы не попасть в нелепую и курьёзную историю. Но даже неудачники мечтают о любви, и наш герой — не исключение, ведь оптимизма ему не занимать.","Гоша, 2019, Россия"


In [31]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


#  Base userkNN model  CV

Compare implicit `CosineRecommender` and `TFIDFRecommender` as an ItemKnn base



In [6]:
# setting for cv
n_folds = 7
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-06-27 00:00:00'), Timestamp('2021-08-22 00:00:00'))


### Test fold borders

In [7]:
periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplitter(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(Interactions(interactions))}")

start_date: 2021-06-27 00:00:00
last_date: 2021-08-22 00:00:00
periods: 8
freq: 1W

Test fold borders: ['2021-06-27' '2021-07-04' '2021-07-11' '2021-07-18' '2021-07-25'
 '2021-08-01' '2021-08-08' '2021-08-15']
Real number of folds: 7


In [12]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10)
}

# few simple models to compare
models = {
#     "BM25": BM25Recommender(),
    "cosine_itemknn": CosineRecommender(),
    "tfidf_itemknn": TFIDFRecommender(),
}


Заметив ошибку в одном из методов определения расстояния, BM25Recommender, мы протестировали его отдельно. К сожалению, не хватило времени разобраться, почему с этим методом возвращается пустой список рекомендаций.

In [16]:
userknn_model = UserKnn(model=BM25Recommender(), N_users=10)
userknn_model.fit(df_train)

recos = userknn_model.predict(df_test)
recos

  0%|          | 0/842129 [00:00<?, ?it/s]

,user_id,item_id,score,rank


# Model training by fold

Здесь мы обучили base_userknn, с дефолтными настройками от лектора.

In [13]:
%%time

results = []

fold_iterator = cv.split(Interactions(interactions), collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        print(model_name)
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)

        recos = userknn_model.predict(df_test)

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )

        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)



==================== Fold 0
{'End date': Timestamp('2021-07-04 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-06-27 00:00:00', freq='W-SUN'),
 'Test': 237414,
 'Test items': 5947,
 'Test users': 98930,
 'Train': 2533586,
 'Train items': 14092,
 'Train users': 536802}
BM25


  0%|          | 0/536802 [00:00<?, ?it/s]

cosine_itemknn


  0%|          | 0/536802 [00:00<?, ?it/s]

tfidf_itemknn


...

cosine_itemknn


  0%|          | 0/842129 [00:00<?, ?it/s]

tfidf_itemknn


  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 5h 46min 31s, sys: 39.4 s, total: 5h 47min 10s
Wall time: 4h 1min 16s


# 👌 Metrics

`Metrics by fold`



## Metric description
- ### <a href="https://rectools.readthedocs.io/en/latest/api/rectools.metrics.novelty.MeanInvUserFreq.html#rectools.metrics.novelty.MeanInvUserFreq"> Mean Inverse User Frequency (novelty)</a>

- ### <a href="https://rectools.readthedocs.io/en/latest/api/rectools.metrics.serendipity.Serendipity.html"> Serendipity = novelty and relevance</a>

In [14]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10
0,0,BM25,0.000000,0.000000,0.000000
1,0,cosine_itemknn,0.004879,0.027282,0.004696
2,0,tfidf_itemknn,0.008546,0.048340,0.008829
3,1,BM25,0.000000,0.000000,0.000000
4,1,cosine_itemknn,0.004807,0.028028,0.004765
5,1,tfidf_itemknn,0.008504,0.050560,0.009345
6,2,BM25,0.000000,0.000000,0.000000
7,2,cosine_itemknn,0.004103,0.023102,0.004082
8,2,tfidf_itemknn,0.006830,0.038295,0.007287
9,3,BM25,0.000000,0.000000,0.000000


In [17]:
# df_metrics.to_pickle("df_metrics.pickle")

## Metrics mean by fold
`we can compare two models`

In [18]:
df_metrics.groupby('model').mean()[metrics.keys()]

,prec@10,recall@10,novelty,serendipity
model,,,,
cosine_itemknn,0.004050,0.022161,8.006506,0.000040
tfidf_itemknn,0.006997,0.038679,7.986296,0.000049


В целом модель с TFIDFRecommender выдает лучшие метрики на кроссвалидации, кроме новизны. Но в новизне разница не сильно большая

## Metrics std by fold

`If a diff between model metrics less than an std value => there is no significant difference observed`

- For instance, for the serendipity metric there is no such difference between cosine_itemknn and tfidf_itemknn model results

In [19]:
df_metrics.groupby('model').std()[metrics.keys()]

,prec@10,recall@10,novelty,serendipity
model,,,,
cosine_itemknn,0.000589,0.004139,0.159162,0.000006
tfidf_itemknn,0.001085,0.007707,0.131361,0.000009


In [20]:
df = df_metrics.groupby('model').mean()[metrics.keys()]

In [21]:
diff = df.loc['cosine_itemknn'] - df.loc['tfidf_itemknn']
diff

prec@10       -0.002946
recall@10     -0.016518
novelty        0.020210
serendipity   -0.000009
dtype: float64

## Decrease N_users

Поскольку TFIDFRecommender выдает в общем лучше метрики в сравнении с косинусным расстоянием в модели, то мы оставим только первый вариант и проведем обучение выбирая разное количество юзеров.

In [17]:
models = {
    "tfidf_itemknn": TFIDFRecommender(),
}
n_users = {
    "n_30": 30,
    "n_20": 20,
    "n_18": 18,
    "n_15": 15,
    "n_10": 10,
}
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

In [18]:
# setting for cv
n_folds = 7
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-06-27 00:00:00'), Timestamp('2021-08-22 00:00:00'))


In [19]:
periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplitter(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(Interactions(interactions))}")

start_date: 2021-06-27 00:00:00
last_date: 2021-08-22 00:00:00
periods: 8
freq: 1W

Test fold borders: ['2021-06-27' '2021-07-04' '2021-07-11' '2021-07-18' '2021-07-25'
 '2021-08-01' '2021-08-08' '2021-08-15']
Real number of folds: 7


In [20]:
%%time

results = []

fold_iterator = cv.split(Interactions(interactions), collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        for name, n in n_users.items():
            print(model_name, name)
            userknn_model = UserKnn(model=model, N_users=n)
            userknn_model.fit(df_train)

            recos = userknn_model.predict(df_test)

            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )

            fold = {"fold": i_fold, "model": model_name, "n_users": n}
            fold.update(metric_values)
            results.append(fold)



==================== Fold 0
{'End date': Timestamp('2021-07-04 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-06-27 00:00:00', freq='W-SUN'),
 'Test': 237414,
 'Test items': 5947,
 'Test users': 98930,
 'Train': 2533586,
 'Train items': 14092,
 'Train users': 536802}
tfidf_itemknn n_30


  0%|          | 0/536802 [00:00<?, ?it/s]

tfidf_itemknn n_20


  0%|          | 0/536802 [00:00<?, ?it/s]

tfidf_itemknn n_18


...

tfidf_itemknn n_15


  0%|          | 0/842129 [00:00<?, ?it/s]

tfidf_itemknn n_10


  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 9h 42min 43s, sys: 54.6 s, total: 9h 43min 38s
Wall time: 6h 20min 3s


In [21]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,n_users,prec@10,recall@10,map@10,novelty,serendipity
0,0,tfidf_itemknn,30,0.008546,0.048340,0.008829,7.799313,0.000035
1,0,tfidf_itemknn,20,0.008546,0.048340,0.008829,7.799313,0.000035
2,0,tfidf_itemknn,18,0.008463,0.047872,0.008708,7.771076,0.000033
3,0,tfidf_itemknn,15,0.008307,0.046835,0.008635,7.740356,0.000032
4,0,tfidf_itemknn,10,0.007263,0.040512,0.007635,7.722912,0.000028
5,1,tfidf_itemknn,30,0.008504,0.050560,0.009345,7.827066,0.000039
6,1,tfidf_itemknn,20,0.008504,0.050560,0.009345,7.827066,0.000039
7,1,tfidf_itemknn,18,0.008417,0.050251,0.009312,7.796292,0.000038
8,1,tfidf_itemknn,15,0.008265,0.049447,0.009086,7.754475,0.000036
9,1,tfidf_itemknn,10,0.007690,0.046020,0.008556,7.710795,0.000029


In [22]:
df_metrics.groupby('n_users').mean()[metrics.keys()]

,prec@10,recall@10,novelty,serendipity,map@10
n_users,,,,,
10,0.006158,0.033868,7.861058,0.000039,0.006400
15,0.006754,0.037299,7.914509,0.000044,0.006995
18,0.006924,0.038286,7.956618,0.000047,0.007189
20,0.006997,0.038679,7.986296,0.000049,0.007266
30,0.006997,0.038679,7.986296,0.000049,0.007266


Как видно, метрики не особо меняются когда юзеров становится больше 20. 

## Changing time split

Попробуем поменять временной промежуток в тесте и посмотреть будут ли отличаться результаты. Оставим пограничное количество юзеров.

In [33]:
n_users = {
    "n_20": 20,
    "n_15": 15,
}

In [34]:
# setting for cv
n_folds = 5
unit = "D"
n_units = 5

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)
print(f"Start date and last date of the test fold: {start_date, last_date}")

periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplitter(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(Interactions(interactions))}")

Start date and last date of the test fold: (Timestamp('2021-07-27 00:00:00'), Timestamp('2021-08-22 00:00:00'))
start_date: 2021-07-27 00:00:00
last_date: 2021-08-22 00:00:00
periods: 6
freq: 5D

Test fold borders: ['2021-07-27' '2021-08-01' '2021-08-06' '2021-08-11' '2021-08-16'
 '2021-08-21']
Real number of folds: 5


In [35]:
%%time

results = []

fold_iterator = cv.split(Interactions(interactions), collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        for name, n in n_users.items():
            print(model_name, name)
            userknn_model = UserKnn(model=model, N_users=n)
            userknn_model.fit(df_train)

            recos = userknn_model.predict(df_test)

            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )

            fold = {"fold": i_fold, "model": model_name, "n_users": n}
            fold.update(metric_values)
            results.append(fold)



==================== Fold 0
{'End date': Timestamp('2021-08-01 00:00:00', freq='5D'),
 'Start date': Timestamp('2021-07-27 00:00:00', freq='5D'),
 'Test': 186642,
 'Test items': 6088,
 'Test users': 77743,
 'Train': 3941089,
 'Train items': 15109,
 'Train users': 749447}
tfidf_itemknn n_20


  0%|          | 0/749447 [00:00<?, ?it/s]

tfidf_itemknn n_15


  0%|          | 0/749447 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-08-06 00:00:00', freq='5D'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='5D'),
 'Test': 193921,
 'Test items': 6096,
 'Test users': 80069,
 'Train': 4203885,
 'Train items': 15212,
 'Train users': 788721}
tfidf_itemknn n_20


...


==================== Fold 4
{'End date': Timestamp('2021-08-21 00:00:00', freq='5D'),
 'Start date': Timestamp('2021-08-16 00:00:00', freq='5D'),
 'Test': 207727,
 'Test items': 6109,
 'Test users': 85168,
 'Train': 5051815,
 'Train items': 15577,
 'Train users': 906071}
tfidf_itemknn n_20


  0%|          | 0/906071 [00:00<?, ?it/s]

tfidf_itemknn n_15


  0%|          | 0/906071 [00:00<?, ?it/s]

CPU times: user 5h 4min 37s, sys: 28 s, total: 5h 5min 5s
Wall time: 3h 17min 45s


In [36]:
df_metrics = pd.DataFrame(results)
df_metrics.groupby('n_users').mean()[metrics.keys()]

,prec@10,recall@10,novelty,serendipity,map@10
n_users,,,,,
15,0.005364,0.029937,8.096305,0.000051,0.005658
20,0.005586,0.031402,8.174692,0.000055,0.005934


In [37]:
# setting for cv
n_folds = 5
unit = "D"
n_units = 4

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)
print(f"Start date and last date of the test fold: {start_date, last_date}")

periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplitter(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(Interactions(interactions))}")

Start date and last date of the test fold: (Timestamp('2021-08-01 00:00:00'), Timestamp('2021-08-22 00:00:00'))
start_date: 2021-08-01 00:00:00
last_date: 2021-08-22 00:00:00
periods: 6
freq: 4D

Test fold borders: ['2021-08-01' '2021-08-05' '2021-08-09' '2021-08-13' '2021-08-17'
 '2021-08-21']
Real number of folds: 5


In [38]:
%%time

results = []

fold_iterator = cv.split(Interactions(interactions), collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        for name, n in n_users.items():
            print(model_name, name)
            userknn_model = UserKnn(model=model, N_users=n)
            userknn_model.fit(df_train)

            recos = userknn_model.predict(df_test)

            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )

            fold = {"fold": i_fold, "model": model_name, "n_users": n}
            fold.update(metric_values)
            results.append(fold)



==================== Fold 0
{'End date': Timestamp('2021-08-05 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='4D'),
 'Test': 160895,
 'Test items': 5785,
 'Test users': 70543,
 'Train': 4203885,
 'Train items': 15212,
 'Train users': 788721}
tfidf_itemknn n_20


  0%|          | 0/788721 [00:00<?, ?it/s]

tfidf_itemknn n_15


  0%|          | 0/788721 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-08-09 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-08-05 00:00:00', freq='4D'),
 'Test': 165639,
 'Test items': 5833,
 'Test users': 72243,
 'Train': 4423600,
 'Train items': 15331,
 'Train users': 819309}
tfidf_itemknn n_20


...


==================== Fold 4
{'End date': Timestamp('2021-08-21 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-08-17 00:00:00', freq='4D'),
 'Test': 171074,
 'Test items': 5824,
 'Test users': 74740,
 'Train': 5106361,
 'Train items': 15589,
 'Train users': 913604}
tfidf_itemknn n_20


  0%|          | 0/913604 [00:00<?, ?it/s]

tfidf_itemknn n_15


  0%|          | 0/913604 [00:00<?, ?it/s]

CPU times: user 6h 15min 11s, sys: 37.7 s, total: 6h 15min 49s
Wall time: 4h 3min 46s


In [39]:
df_metrics = pd.DataFrame(results)
df_metrics.groupby('n_users').mean()[metrics.keys()]

,prec@10,recall@10,novelty,serendipity,map@10
n_users,,,,,
15,0.005039,0.029364,8.163233,0.000050,0.005543
20,0.005236,0.030714,8.241690,0.000054,0.005815


Поскольку модель userknn не дает рекомендаций для холодных и теплых польхователей, то мы добавили модель, возвращающая топ популярных. И метрика map@10 сильно выросла на сервисе. Оказалось сложно сравнить метрики оффлайн и онлайн в таком случае. Но так как, при количестве пользователей 20 и выше метрики на оффлайн не меняются, то именно это значение и оставили.